<a href="https://colab.research.google.com/github/ParthaSarathiChakraborty/equity_research/blob/main/Volatility_Driven_Asset_Allocation_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Volatility-Driven Asset Allocation Strategy
This notebook implements a tactical asset allocation strategy using the VIX index to switch between equities and bonds based on market volatility.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

# Install seaborn if not already installed
!pip install seaborn

# Import seaborn to ensure its styles are registered with matplotlib
import seaborn as sns

# Now, using the 'seaborn' style should work
plt.style.use('seaborn-v0_8') # Use the updated seaborn style name

## 📥 Download Data
We download historical data for SPY (equities), AGG (bonds), and VIX (volatility index) from Yahoo Finance.

In [ ]:
start_date = '2018-01-01'
end_date = '2024-01-01'

spy_data = yf.download('SPY', start=start_date, end=end_date)
print(spy_data.columns) # Examine this output to find the correct adjusted close column name!

# Use the correct column name as identified from the print output.
# **IMPORTANT**: Replace 'Adj Close' below with the actual column name
# for adjusted closing price from the print output above if it's different.
try:
    spy = spy_data['Adj Close']
except KeyError:
    # Fallback if 'Adj Close' is not found - try 'Close' or 'AdjClose' based on typical yfinance outputs
    try:
        spy = spy_data['Close']
    except KeyError:
        try:
            spy = spy_data['AdjClose'] # Trying the original name just in case of variations
        except KeyError:
             print("Could not find 'Adj Close', 'Close', or 'AdjClose' columns in SPY data.")
             # You might want to handle this error further, e.g., exit or raise an exception
             raise # Re-raise the last exception

# Apply similar logic for AGG and VIX if they also might have unexpected column names
try:
    agg = yf.download('AGG', start=start_date, end=end_date)['Adj Close']
except KeyError:
    try:
        agg = yf.download('AGG', start=start_date, end=end_date)['Close']
    except KeyError:
         try:
            agg = yf.download('AGG', start=start_date, end=end_date)['AdjClose']
         except KeyError:
             print("Could not find 'Adj Close', 'Close', or 'AdjClose' columns in AGG data.")
             raise


try:
    vix = yf.download('^VIX', start=start_date, end=end_date)['Adj Close']
except KeyError:
     try:
        vix = yf.download('^VIX', start=start_date, end=end_date)['Close']
     except KeyError:
         try:
            vix = yf.download('^VIX', start=start_date, end=end_date)['AdjClose']
         except KeyError:
             print("Could not find 'Adj Close', 'Close', or 'AdjClose' columns in VIX data.")
             raise

## 🔧 Prepare Data
Merge all series and forward-fill missing values.

In [ ]:
df = pd.concat([spy, agg, vix], axis=1)
df.columns = ['SPY', 'AGG', 'VIX']
df = df.fillna(method='ffill')

# Create signals: 1 if VIX <= 20, else 0 (equity or bond)
df['Signal'] = np.where(df['VIX'] <= 20, 1, 0)

## 🧠 Strategy Logic
If VIX <= 20: Invest in SPY, else switch to AGG.

In [ ]:
df['SPY_Returns'] = df['SPY'].pct_change()
df['AGG_Returns'] = df['AGG'].pct_change()
df['Strategy_Returns'] = df['Signal'].shift(1) * df['SPY_Returns'] + (1 - df['Signal'].shift(1)) * df['AGG_Returns']
df.dropna(inplace=True)

## 📈 Cumulative Performance Comparison

In [ ]:
(1 + df[['SPY_Returns', 'AGG_Returns', 'Strategy_Returns']]).cumprod().plot(figsize=(12, 6))
plt.title('Cumulative Returns: SPY vs AGG vs Strategy')
plt.ylabel('Growth of $1')
plt.grid(True)
plt.show()

## 📊 Performance Metrics

In [ ]:
performance = pd.DataFrame({
    'Annual Return': df[['SPY_Returns', 'AGG_Returns', 'Strategy_Returns']].mean() * 252,
    'Volatility': df[['SPY_Returns', 'AGG_Returns', 'Strategy_Returns']].std() * np.sqrt(252),
    'Sharpe Ratio': (df[['SPY_Returns', 'AGG_Returns', 'Strategy_Returns']].mean() / df[['SPY_Returns', 'AGG_Returns', 'Strategy_Returns']].std()) * np.sqrt(252)
})
performance.columns = ['Annual Return', 'Volatility', 'Sharpe Ratio']
print(performance.round(3))